# Baum-Welch Algorithm

In [34]:
# Imports
import numpy as np

# Load Biodata

In [35]:
!curl -O https://services.healthtech.dtu.dk/services/TMHMM-2.0/TMHMM/data/set160.labels
!pip install biopython

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  126k  100  126k    0     0   133k      0 --:--:-- --:--:-- --:--:--  132k


In [36]:
from Bio import SeqIO
records = list(SeqIO.parse("set160.labels", "fasta"))

sequence_list = []
target_seqs = []

for record in records:
  input_seq, target_seq = record.seq.split("#")
  sequence_list.append(str(input_seq))
  target_seqs.append(str(target_seq))

# Define functions

In [37]:
# Do an integer encoding of input sequence
def encode( sequence, symbols):
    
    enc = [0] * len(sequence)
    
    for i in range(len(sequence)):
        enc[i] = symbols.find(sequence[i])
    
    return(enc)

In [38]:
# Function for scaling
def normalize(v):
    norm = np.sum(v)
    v = v/norm
    return (v, norm)

## Forward Loop

In [39]:
def initialize_forward(input_encode, states, initial_prob, emission_probs):
    
    alpha = np.zeros(shape=(states, len(input_encode)))
        
    for i in range(0, states): 
        
        alpha[i][0] = initial_prob[i]*emission_probs[i][input_encode[0]]
        
    return alpha

In [40]:
def run_forward(input_encode, states, initial_prob, emission_probs):

    alpha = initialize_forward(input_encode, states, initial_prob, emission_probs)
    norm_vector = np.zeros(len(input_encode))
    alpha[:,0], norm_vector[0] = normalize(alpha[:,0])


    for i in range(1, len(input_encode)):
        
        for j in range(0, states):

            _sum = 0
            
            for k in range(0, states):
                
                _sum += alpha[k][i-1] * transition_matrix[k,j] 
      
            # store prob
            alpha[j][i] = emission_probs[j][input_encode[i]] * _sum

        # Normalize
        alpha[:,i],norm_vector[i] = normalize(alpha[:,i])

    return alpha, norm_vector

## Backward Loop

In [41]:
def initialize_backward(input_encode, states):
  
    
    beta = np.zeros(shape=(states, len(input_encode)))
        
    for i in range(0, states):
  
        beta[i][-1] = 1
        
    return beta

In [42]:
def run_backward(input_encode, states, emission_probs, transition_matrix, norm_vector):

    beta = initialize_backward(input_encode, states)

    for t in range(len(input_encode)-2, -1 , -1):
      for i in range(states):

            _sum = 0

            for j in range(states):
              _sum += beta[j][t+1] * transition_matrix[i][j] * emission_probs[j][input_encode[t+1]]
                
            # store prob
            beta[i][t] = _sum

      beta[:,t] = beta[:,t] / norm_vector[t]

    return beta

##Load sequences

In [43]:
symbols = 'DCILTPGRYQWNFHKMVEAS'

# Encode sequences
encoded_list = []
max_seq_length = 0
for input_sequence in sequence_list:
  encoded_list.append(encode(input_sequence, symbols))
  if len(input_sequence) > max_seq_length:
    max_seq_length = len(input_sequence)

V = len(symbols)
num_sequence = len(encoded_list)

##Implementaion of Baum-Welsh

In [44]:
states = 4 # Expect: 0= iC, 1 = Membrane, 2 = eC, 3 = Membrane
num_iters = 100

# Chose initial model
initial_prob = np.array([0.5, 0.0, 0.5, 0.0])

transition_matrix = np.array([[9/10, 1/10, 0.0, 0.0], [0.0, 24/25, 1/25, 0.0], [0.0, 0.0, 9/10, 1/10], [1/25, 0.0, 0.0, 24/25]])

# Initial guess
# All aminoacids have the same prob (could be changed to background freqs)
state1_guess = np.repeat(1/V, V)
state2_guess = np.repeat(1/V, V)
state3_guess = np.repeat(1/V, V)
state4_guess = np.repeat(1/V, V)

emission_probs = [state1_guess, state2_guess, state3_guess, state4_guess]

for iteration in range(num_iters):
  print('Starting iteration ', iteration)
    
  # Initiate xi
  xi = np.zeros(max_seq_length*states*states*num_sequence).reshape(states,states,max_seq_length,num_sequence)

  gamma = np.zeros(max_seq_length*states*num_sequence).reshape(states,max_seq_length,num_sequence)
  gamma_sums = np.zeros(max_seq_length*states*num_sequence).reshape(states,max_seq_length,num_sequence)

  for r in range(num_sequence):

    # Load and encode sequence
    input_encode = encoded_list[r]
    T = len(input_encode)

    # Calculate alpha and beta matrix
    alpha, norm_vector = run_forward(input_encode, states, initial_prob, emission_probs)
    beta = run_backward(input_encode, states, emission_probs, transition_matrix, norm_vector)

    for i in range(states):
      for t in range(T-1):
        for j in range(states):

          # Calculate denominator
          gamma_sums[i][t][r] += alpha[j][t] * beta[j][t]
          # Calculate Xi
          
          # Summation of denominator of xi
          xi_denominator = 0

          for k in range(states):
            for w in range(states):

              # check that transition matrix is defined correct?
              xi_denominator += alpha[k][t] * transition_matrix[k][w] * beta[w][t+1] * emission_probs[w][input_encode[t+1]]

          # Calculate xi
          xi[i][j][t][r] = (alpha[i][t] * transition_matrix[i][j] * beta[j][t+1] * emission_probs[j][input_encode[t+1]]) / xi_denominator # Check this

        gamma[i][t][r] = (alpha[i][t] * beta[i][t]) / gamma_sums[i][t][r]


  # Update of parameters
  for i in range(states):

    # Update initial probabilities
    initial_prob[i] = 0
    for r in range(num_sequence):
      initial_prob[i] += gamma[i][1][r] / num_sequence

    for j in range(states):

      # Update transition matrix
      xi_sum = 0
      gamma_sum = 0

      for r in range(num_sequence):
        T = len(encoded_list[r]) 
        for t in range(T-1):
        
          xi_sum += xi[i][j][t][r]
          gamma_sum += gamma[i][t][r]
      
      transition_matrix[i][j] = xi_sum / gamma_sum

      # Update emission probs
      for k in range(V):
        denominator_sum = 0
        numerator_sum = 0
        for r in range(num_sequence):
          T = len(encoded_list[r]) 
          for t in range(T):
          
            denominator_sum += gamma[i][t][r]
        
            if encoded_list[r][t] == k:
              numerator_sum += gamma[i][t][r]
          
        emission_probs[i][k] = numerator_sum / denominator_sum

  
  # Normalize transition
  for row in range(states):
    summ = sum(transition_matrix[row])
    for col in range(states):
      transition_matrix[row][col] = transition_matrix[row][col] / summ
  
  # Normalize emission
  for row in range(states):
    summ = sum(emission_probs[row])
    for col in range(V):
      emission_probs[row][col] = emission_probs[row][col] / summ  

print('iteration: ', iteration)
print('Initial probabilities: ')
print(initial_prob)
print('Normalized transition matrix')
print(transition_matrix)
print('Normalized Emission probabilities')
print(emission_probs)
      

Starting iteration  0
Starting iteration  1
Starting iteration  2
Starting iteration  3
Starting iteration  4
Starting iteration  5
Starting iteration  6
Starting iteration  7
Starting iteration  8
Starting iteration  9
Starting iteration  10
Starting iteration  11
Starting iteration  12
Starting iteration  13
Starting iteration  14
Starting iteration  15
Starting iteration  16
Starting iteration  17
Starting iteration  18
Starting iteration  19
Starting iteration  20
Starting iteration  21
Starting iteration  22
Starting iteration  23
Starting iteration  24
Starting iteration  25
Starting iteration  26
Starting iteration  27
Starting iteration  28
Starting iteration  29
Starting iteration  30
Starting iteration  31
Starting iteration  32
Starting iteration  33
Starting iteration  34
Starting iteration  35
Starting iteration  36
Starting iteration  37
Starting iteration  38
Starting iteration  39
Starting iteration  40
Starting iteration  41
Starting iteration  42
Starting iteration  4

KeyboardInterrupt: ignored

In [45]:
print('iteration: ', iteration)
print('Initial probabilities: ')
print(initial_prob)
print('Normalized transition matrix')
print(transition_matrix)
print('Normalized Emission probabilities')
print(emission_probs)

iteration:  50
Initial probabilities: 
[0.14439263 0.35560737 0.14439263 0.35560737]
Normalized transition matrix
[[0.94770004 0.05229996 0.         0.        ]
 [0.         0.97857193 0.02142807 0.        ]
 [0.         0.         0.94770004 0.05229996]
 [0.02142807 0.         0.         0.97857193]]
Normalized Emission probabilities
[array([0.01029153, 0.01908291, 0.10631847, 0.15678684, 0.05062867,
       0.03221489, 0.09260626, 0.01123382, 0.03736221, 0.00938103,
       0.027464  , 0.02032179, 0.08649205, 0.00934898, 0.00978675,
       0.04142035, 0.10402677, 0.00949588, 0.10868485, 0.05705196]), array([0.05531167, 0.02006601, 0.04584805, 0.08552509, 0.06001331,
       0.05545439, 0.06466026, 0.06295773, 0.03449989, 0.04531023,
       0.01624969, 0.04823913, 0.03642936, 0.02305122, 0.05703138,
       0.02345883, 0.06187132, 0.06564297, 0.06760485, 0.07077462]), array([0.01029153, 0.01908291, 0.10631847, 0.15678684, 0.05062867,
       0.03221489, 0.09260626, 0.01123382, 0.03736221, 

# Viterbi - Run on one sequence

In [58]:
# Define new sequence
input_sequence = sequence_list[0]

In [59]:
def initialize(encode_sequence, states, initial_prob, transition_matrix, emission_probs):
    
    delta = np.zeros(shape=(states, len(encode_sequence)))
    
    arrows = np.ndarray(shape=(states, len(encode_sequence)), dtype=object)
    
    # initial conditions
    for i in range(0, states):

	## delta[i][0] = XX  
        delta[i][0] = initial_prob[i] + emission_probs[i][encode_sequence[0]]
    
        arrows[i][0] = 0
    
    return delta, arrows

In [60]:
input_encode = encode(input_sequence, symbols)

delta, arrows = initialize(input_encode, states, initial_prob, transition_matrix, emission_probs)

# main loop
for i in range(1, len(input_sequence)):
    
    for j in range(0, states):
        
        max_arrow_prob = -np.inf
        max_arrow_prob_state = -1
        
        for k in range(0, states):
           
	    # arrow_prob = XX 
            arrow_prob = delta[k][i-1] + transition_matrix[k, j]
            
            if arrow_prob > max_arrow_prob: 
		# max_arrow_prob = XX
                max_arrow_prob = arrow_prob
                # max_arrow_prob_state = XX
                max_arrow_prob_state = k
            
        # store prob
	# delta[j][i] = XX + max_arrow_prob
        delta[j][i] = emission_probs[j][input_encode[i]] + max_arrow_prob

        # store arrow
        arrows[j][i] = max_arrow_prob_state

print(delta)


[[1.85812982e-01 1.24219787e+00 2.19968466e+00 ... 6.64096927e+02
  6.65054918e+02 6.66050833e+02]
 [3.79066191e-01 1.42524297e+00 2.46084628e+00 ... 6.64985735e+02
  6.66019618e+02 6.67055222e+02]
 [1.85812982e-01 1.24219787e+00 2.19968466e+00 ... 6.64096927e+02
  6.65054918e+02 6.66050833e+02]
 [3.79066191e-01 1.42524297e+00 2.46084628e+00 ... 6.64985735e+02
  6.66019618e+02 6.67055222e+02]]


In [61]:
path = []

max_state = np.argmax(delta[:, -1])
max_value = delta[max_state, -1]

print ("log(Max_path):", max_value)

print ("Seq: ", input_sequence)

path.append(str(max_state))

old_state = max_state

for i in range(len(input_encode)-2, -1, -1):
    
    # current_state = arrows[XX][i+1]
    current_state = arrows[old_state][i+1]
    
    path.append(str(current_state))

    old_state = current_state 
    
path_str = "".join(reversed(path)).replace('0','i').replace('1','M').replace('2','o').replace('3','M')

print ("Path:", path_str)

log(Max_path): 667.0552216790004
Seq:  MAKNLILWLVIAVVLMSVFQSFGPSESNGRKVDYSTFLQEVNNDQVREARINGREINVTKKDSNRYTTYIPVQDPKLLDNLLTKNVKVVGEPPEEPSLLASIFISWFPMLLLIGVWIFFMRQMQGGGGKGAMSFGKSKARMLTEDQIKTTFADVAGCDEAKEEVAELVEYLREPSRFQKLGGKIPKGVLMVGPPGTGKTLLAKAIAGEAKVPFFTISGSDFVEMFVGVGASRVRDMFEQAKKAAPCIIFIDEIDAVGRQRGAGLGGGHDEREQTLNQMLVEMDGFEGNEGIIVIAATNRPDVLDPALLRPGRFDRQVVVGLPDVRGREQILKVHMRRVPLAPDIDAAIIARGTPGFSGADLANLVNEAALFAARGNKRVVSMVEFEKAKDKIMMGAERRSMVMTEAQKESTAYHEAGHAIIGRLVPEHDPVHKVTIIPRGRALGVTFFLPEGDAISASRQKLESQISTLYGGRLAEEIIYGPEHVSTGASNDIKVATNLARNMVTQWGFSEKLGPLLYAEEEGEVFLGRSVAKAKHMSDETARIIDQEVKALIERNYNRARQLLTDNMDILHAMKDALMKYETIDAPQIDDLMARRDVRPPAGWEEPGASNNSGDNGSPKAPRPVDEPRTPNPGNTMSEQLGDK
Path: MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM

In [62]:
target_sequence = target_seqs[0]
print(target_sequence)
print(path_str)

iiiiMMMMMMMMMMMMMMMMMMMMoooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooMMMMMMMMMMMMMMMMMMMMMMMMMiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM